In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
from graphviz import Digraph

In [7]:
class Value:
    
    def __init__(self, data, _children = (), _op = '', mark = ''):
        self.data = data
        self.grad = 0.0
        self.kids = _children
        self.actn = _op
        self.mark = mark

    def __repr__(self):
        return f"Value({self.data})"
    
    def __add__(self, other):
        return Value(self.data + other.data, ( self, other ), '+')

    def __sub__(self, other):
        return Value(self.data - other.data, ( self, other ), '-')

    def __mul__(self, other):
        return Value(self.data * other.data, ( self, other ), '*')

a = Value(2.0, mark = 'a')
b = Value(3.0, mark = 'b')
c = Value(4.0, mark = 'c')
d = a*b + c
d

Value(10.0)